#                 Initialize

In [13]:
import pandas as pd
import os
import numpy as np
# from IPython.display import display, Markdown

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()

In [12]:
# directories
working_directory   = os.getcwd()
data_folder         = os.path.join(working_directory, "data"         )
intermediate_folder = os.path.join(working_directory, "intermediate" )
results_folder      = os.path.join(working_directory, "results"      )
webapplayers_folder = os.path.join(results_folder   , r"webapplayers" )
webapplayersshps_folder = os.path.join(results_folder   , r"webapplayers\shapefiles" )
webapplayerszips_folder = os.path.join(results_folder   , r"webapplayers\shapefiles_zips" )

centers_smalldist_shp = os.path.join(data_folder, 'centers_smalldist_shp\\zone_set_WasatchChoiceCenters_WFSmallAnalysisDists.shp')

csvCentersSmallDist = os.path.join(intermediate_folder, "centers_small_districts_od_by_mode.csv")
dfCentersSmallDistOD  = pd.read_csv(csvCentersSmallDist)

dfModes = pd.DataFrame({'mode'      :['ALL','RAI' ,'BUS','BIC'    ,'PED'       ,'TRU'  ],
                        'mode_label':['All','Rail','Bus','Bicycle','Pedestrian','Truck']})
display(dfModes)

# big data field names
#fnCounts = 'o_d_traffic_sample_trip_counts'
fnVolume = 'odvolume'

# show numbers with commas
pd.options.display.float_format = '{:,.0f}'.format

,mode,mode_label
0,ALL,All
1,RAI,Rail
2,BUS,Bus
3,BIC,Bicycle
4,PED,Pedestrian
5,TRU,Truck


In [11]:
dfDateRanges = dfCentersSmallDistOD.groupby(['date_range'],as_index=False).agg(NUMREC=('OID','count'))
dfDateRanges['year'  ] = dfDateRanges['date_range'].str[:4 ].astype(int)
dfDateRanges['season'] = dfDateRanges['date_range'].str[-2:]
display(dfDateRanges)

dfYears = dfDateRanges.groupby(['year'],as_index=False).agg(NUMREC=('NUMREC','count'))
dfYears['value'] = dfYears['year']
dfYears['label'] = dfYears['year']
display(dfYears)

dfSeasons = pd.DataFrame({'value':['YR'         ,'01'     ,'02'      ,'03'   ,'04'   ,'05' ,'06'  ,'07'  ,'08'    ,'09'       ,'10'     ,'11'      ,'12'      ,'BM'         ],
                          'label':['Entire Year','January','February','March','April','May','June','July','August','September','October','November','December','Bike Months']})
display(dfSeasons)


,date_range,NUMREC,year,season
0,2019_01,281528,2019,01
1,2019_02,278946,2019,02
2,2019_03,295986,2019,03
3,2019_04,337322,2019,04
4,2019_05,341743,2019,05
5,2019_06,314328,2019,06
6,2019_07,312599,2019,07
7,2019_08,315693,2019,08
8,2019_09,347835,2019,09
9,2019_10,343787,2019,10


,year,NUMREC,value,label
0,2019,14,2019,2019
1,2020,14,2020,2020
2,2021,14,2021,2021


,value,label
0,YR,Entire Year
1,01,January
2,02,February
3,03,March
4,04,April
5,05,May
6,06,June
7,07,July
8,08,August
9,09,September


In [16]:
sdfCentersSmallDist = pd.DataFrame.spatial.from_featureclass(centers_smalldist_shp)
#sdfCentersSmallDist
dfCSD = sdfCentersSmallDist[['id','group']]
dfCSD

,id,group
0,11,smalldist
1,21,smalldist
2,31,smalldist
3,32,smalldist
4,41,smalldist
...,...,...
133,"2,010",center
134,"2,011",center
135,"2,012",center
136,"2,013",center


In [56]:
dfCenterJson = sdfCentersSmallDist[sdfCentersSmallDist['group']=='center'].copy()
dfCenterJson['value'] = dfCenterJson['id'].astype(int)
dfCenterJson['label'] = dfCenterJson['name']
dfCenterJson = dfCenterJson[['value','label']]
display(dfCenterJson)
dfCenterJson.to_json(os.path.join('results',r'data\centers.json'),orient='records')


,value,label
123,2000,Downtown Provo
124,2001,Thanksgiving Point
125,2002,Layton Midtown
126,2003,The District
127,2004,9th and 9th Center
128,2005,Sugarhouse
129,2006,Murray Downtown
130,2007,The Cairns
131,2008,West Jordan City Center
132,2009,Fairbourne Station


In [21]:
dfCentersSmallDistOD_wOGroups = pd.DataFrame.merge(dfCentersSmallDistOD,dfCSD,left_on='OID',right_on='id')
dfCentersSmallDistOD_wOGroups = dfCentersSmallDistOD_wOGroups.drop(columns='id')
dfCentersSmallDistOD_wOGroups = dfCentersSmallDistOD_wOGroups.rename(columns={'group':'ogroup'})
dfCentersSmallDistOD_wOGroups

,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,ogroup
0,101,101,0,0,15720,770,TRU,2019_01,smalldist
1,101,101,0,1,984,812,TRU,2019_01,smalldist
2,101,101,0,2,3093,881,TRU,2019_01,smalldist
3,101,101,0,3,7330,743,TRU,2019_01,smalldist
4,101,101,0,4,3056,703,TRU,2019_01,smalldist
...,...,...,...,...,...,...,...,...,...
12924530,662,662,2,5,2,NaN,ALL,2021_YR,smalldist
12924531,662,721,0,0,0,NaN,ALL,2021_YR,smalldist
12924532,662,721,0,3,0,NaN,ALL,2021_YR,smalldist
12924533,662,721,1,0,0,NaN,ALL,2021_YR,smalldist


In [24]:
dfCentersSmallDistOD_wOGroupsDGroups = pd.DataFrame.merge(dfCentersSmallDistOD_wOGroups,dfCSD,left_on='DID',right_on='id')
dfCentersSmallDistOD_wOGroupsDGroups = dfCentersSmallDistOD_wOGroupsDGroups.drop(columns='id')
dfCentersSmallDistOD_wOGroupsDGroups = dfCentersSmallDistOD_wOGroupsDGroups.rename(columns={'group':'dgroup'})
dfCentersSmallDistOD_wOGroupsDGroups


,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,ogroup,dgroup
0,101,101,0,0,15720,770,TRU,2019_01,smalldist,smalldist
1,101,101,0,1,984,812,TRU,2019_01,smalldist,smalldist
2,101,101,0,2,3093,881,TRU,2019_01,smalldist,smalldist
3,101,101,0,3,7330,743,TRU,2019_01,smalldist,smalldist
4,101,101,0,4,3056,703,TRU,2019_01,smalldist,smalldist
...,...,...,...,...,...,...,...,...,...,...
12924530,662,662,2,0,15,"1,199",ALL,2021_YR,smalldist,smalldist
12924531,662,662,2,2,5,NaN,ALL,2021_YR,smalldist,smalldist
12924532,662,662,2,3,4,NaN,ALL,2021_YR,smalldist,smalldist
12924533,662,662,2,4,3,NaN,ALL,2021_YR,smalldist,smalldist


In [37]:
dfCentersSmallDistOD_wOGroupsDGroups['group'] = ''
dfCentersSmallDistOD_wOGroupsDGroups.loc[(dfCentersSmallDistOD_wOGroupsDGroups['ogroup']=='smalldist') & (dfCentersSmallDistOD_wOGroupsDGroups['dgroup']=='smalldist'),'group'] = 'smalldist'
dfCentersSmallDistOD_wOGroupsDGroups.loc[(dfCentersSmallDistOD_wOGroupsDGroups['ogroup']=='center'   ) & (dfCentersSmallDistOD_wOGroupsDGroups['dgroup']=='smalldist'),'group'] = 'centertosmalldist'
dfCentersSmallDistOD_wOGroupsDGroups.loc[(dfCentersSmallDistOD_wOGroupsDGroups['ogroup']=='smalldist') & (dfCentersSmallDistOD_wOGroupsDGroups['dgroup']=='center'   ),'group'] = 'centertosmalldist'
dfCentersSmallDistOD_wGroups = dfCentersSmallDistOD_wOGroupsDGroups.drop(columns=['ogroup','dgroup'])

# only include rows with groups
dfCentersSmallDistOD_wGroups = dfCentersSmallDistOD_wGroups[dfCentersSmallDistOD_wGroups['group']!='']
dfCentersSmallDistOD_wGroups

,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,group
0,101,101,0,0,15720,770.0000,TRU,2019_01,smalldist
1,101,101,0,1,984,812.0000,TRU,2019_01,smalldist
2,101,101,0,2,3093,881.0000,TRU,2019_01,smalldist
3,101,101,0,3,7330,743.0000,TRU,2019_01,smalldist
4,101,101,0,4,3056,703.0000,TRU,2019_01,smalldist
...,...,...,...,...,...,...,...,...,...
12924530,662,662,2,0,15,"1,199.0000",ALL,2021_YR,smalldist
12924531,662,662,2,2,5,NaN,ALL,2021_YR,smalldist
12924532,662,662,2,3,4,NaN,ALL,2021_YR,smalldist
12924533,662,662,2,4,3,NaN,ALL,2021_YR,smalldist


In [38]:
dfCentersSmallDistOD_wGroups[dfCentersSmallDistOD_wGroups['group']=='centertosmalldist']

,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,group
26377,2000,101,0,0,147,NaN,TRU,2019_01,centertosmalldist
26378,2000,101,0,1,11,NaN,TRU,2019_01,centertosmalldist
26379,2000,101,0,4,84,NaN,TRU,2019_01,centertosmalldist
26380,2000,101,0,5,53,NaN,TRU,2019_01,centertosmalldist
26381,2000,101,1,0,189,NaN,TRU,2019_01,centertosmalldist
...,...,...,...,...,...,...,...,...,...
12920268,2013,662,1,2,0,NaN,ALL,2020_BM,centertosmalldist
12920269,2013,662,0,0,0,NaN,ALL,2020_YR,centertosmalldist
12920270,2013,662,0,2,0,NaN,ALL,2020_YR,centertosmalldist
12920271,2013,662,1,0,0,NaN,ALL,2020_YR,centertosmalldist


In [39]:
dfCentersSmallDistOD_wGroups.groupby('group').agg(NUMREC=('OID','count'))

,NUMREC
group,
centertosmalldist,2495754
smalldist,10243731


## Create Percent Distribution

In [59]:
# remove intrazonals from analysis
dfOD_noIntrazonals = dfCentersSmallDistOD_wGroups.loc[dfCentersSmallDistOD_wGroups['OID']!=dfCentersSmallDistOD_wGroups['DID']]

# create group totals for origin and destination and then find percent breakdown for each
dfOD_noIntrazonals_Orig_GroupTotals = dfOD_noIntrazonals.groupby(['group','OID','day_type','day_part','date_range','mode'],as_index=False).agg(OTotalVolume=(fnVolume,'sum'))
dfOD_noIntrazonals_Dest_GroupTotals = dfOD_noIntrazonals.groupby(['group','DID','day_type','day_part','date_range','mode'],as_index=False).agg(DTotalVolume=(fnVolume,'sum'))
dfOD_noIntrazonals_Orig_withGroupTotals = pd.DataFrame.merge(dfOD_noIntrazonals,dfOD_noIntrazonals_Orig_GroupTotals,on=('group','OID','day_type','day_part','date_range','mode'))
dfOD_noIntrazonals_Orig_withGroupTotals['OPercentTotal'] = round(dfOD_noIntrazonals_Orig_withGroupTotals[fnVolume] / dfOD_noIntrazonals_Orig_withGroupTotals['OTotalVolume'],4)
dfOD_noIntrazonals_Dest_withGroupTotals = pd.DataFrame.merge(dfOD_noIntrazonals,dfOD_noIntrazonals_Dest_GroupTotals,on=('group','DID','day_type','day_part','date_range','mode'))
dfOD_noIntrazonals_Dest_withGroupTotals['DPercentTotal'] = round(dfOD_noIntrazonals_Dest_withGroupTotals[fnVolume] / dfOD_noIntrazonals_Dest_withGroupTotals['DTotalVolume'],4)

dfODSummary = pd.DataFrame.merge(dfOD_noIntrazonals_Orig_withGroupTotals, dfOD_noIntrazonals_Dest_withGroupTotals, on=('group','OID','DID','day_type','day_part','odvolume','odtime','mode','date_range'))

dfODSummary['odvolume'] = dfODSummary['odvolume'].astype(int)

dfODSummary = dfODSummary.fillna(0)

dfODSummary['BPercentTotal'] = (dfODSummary['OPercentTotal'] + dfODSummary['DPercentTotal']) / 2

dfODSummary = dfODSummary[['group','OID','DID','date_range','day_type','day_part','mode','odvolume','OPercentTotal','DPercentTotal','BPercentTotal']]

# show numbers with commas
pd.options.display.float_format = '{:,.8f}'.format

display(dfOD_noIntrazonals_Orig_withGroupTotals)
display(dfOD_noIntrazonals_Dest_withGroupTotals)
display(dfODSummary)

# show numbers with commas
pd.options.display.float_format = '{:,.0f}'.format

,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,group,OTotalVolume,OPercentTotal
0,102,101,0,0,1991,758.00000000,TRU,2019_01,smalldist,7279,0.27350000
1,102,11,0,0,5,NaN,TRU,2019_01,smalldist,7279,0.00070000
2,102,111,0,0,730,"1,345.00000000",TRU,2019_01,smalldist,7279,0.10030000
3,102,112,0,0,58,NaN,TRU,2019_01,smalldist,7279,0.00800000
4,102,121,0,0,1056,"1,019.00000000",TRU,2019_01,smalldist,7279,0.14510000
...,...,...,...,...,...,...,...,...,...,...,...
12401790,661,662,0,3,2,"1,453.00000000",BIC,2019_10,smalldist,2,1.00000000
12401791,661,662,0,0,0,"1,453.00000000",BIC,2019_BM,smalldist,0,NaN
12401792,661,662,0,3,0,"1,453.00000000",BIC,2019_BM,smalldist,0,NaN
12401793,661,662,0,0,0,"1,453.00000000",BIC,2019_YR,smalldist,0,NaN


,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,group,DTotalVolume,DPercentTotal
0,102,101,0,0,1991,758.00000000,TRU,2019_01,smalldist,44973,0.04430000
1,11,101,0,0,74,NaN,TRU,2019_01,smalldist,44973,0.00160000
2,111,101,0,0,2533,"1,312.00000000",TRU,2019_01,smalldist,44973,0.05630000
3,112,101,0,0,147,"2,016.00000000",TRU,2019_01,smalldist,44973,0.00330000
4,121,101,0,0,7644,"1,209.00000000",TRU,2019_01,smalldist,44973,0.17000000
...,...,...,...,...,...,...,...,...,...,...,...
12401790,621,662,1,5,3,278.00000000,PED,2021_YR,smalldist,3,1.00000000
12401791,621,662,2,4,7,938.00000000,PED,2021_YR,smalldist,7,1.00000000
12401792,631,662,1,3,6,NaN,ALL,2020_10,smalldist,6,1.00000000
12401793,691,662,1,3,2,NaN,ALL,2020_11,smalldist,2,1.00000000


,group,OID,DID,date_range,day_type,day_part,mode,odvolume,OPercentTotal,DPercentTotal,BPercentTotal
0,smalldist,102,101,2019_01,0,0,TRU,1991,0.27350000,0.04430000,0.15890000
1,smalldist,102,11,2019_01,0,0,TRU,5,0.00070000,0.00170000,0.00120000
2,smalldist,102,111,2019_01,0,0,TRU,730,0.10030000,0.02400000,0.06215000
3,smalldist,102,112,2019_01,0,0,TRU,58,0.00800000,0.00990000,0.00895000
4,smalldist,102,121,2019_01,0,0,TRU,1056,0.14510000,0.02140000,0.08325000
...,...,...,...,...,...,...,...,...,...,...,...
12401790,smalldist,661,662,2019_10,0,3,BIC,2,1.00000000,1.00000000,1.00000000
12401791,smalldist,661,662,2019_BM,0,0,BIC,0,0.00000000,0.00000000,0.00000000
12401792,smalldist,661,662,2019_BM,0,3,BIC,0,0.00000000,0.00000000,0.00000000
12401793,smalldist,661,662,2019_YR,0,0,BIC,0,0.00000000,0.00000000,0.00000000


In [60]:
dfODSummary_pivotMode_Vol = dfODSummary.pivot_table(index=('group','OID','DID','date_range','day_type','day_part'),values=('odvolume'),columns='mode')
dfODSummary_pivotMode_Vol = dfODSummary_pivotMode_Vol.reset_index()
dfODSummary_pivotMode_Vol = dfODSummary_pivotMode_Vol.fillna(0)
dfODSummary_pivotMode_Vol = dfODSummary_pivotMode_Vol.rename(columns={'OID':'O','DID':'D','ALL':"AV",'BIC':'KV','BUS':'SV','PED':'PV','RAI':'RV','TRU':'TV'})

dfODSummary_pivotMode_Vol['AV'] = dfODSummary_pivotMode_Vol['AV'].astype(int)
dfODSummary_pivotMode_Vol['KV'] = dfODSummary_pivotMode_Vol['KV'].astype(int)
dfODSummary_pivotMode_Vol['SV'] = dfODSummary_pivotMode_Vol['SV'].astype(int)
dfODSummary_pivotMode_Vol['PV'] = dfODSummary_pivotMode_Vol['PV'].astype(int)
dfODSummary_pivotMode_Vol['RV'] = dfODSummary_pivotMode_Vol['RV'].astype(int)
dfODSummary_pivotMode_Vol['TV'] = dfODSummary_pivotMode_Vol['TV'].astype(int)

display(dfODSummary_pivotMode_Vol)

# show numbers with commas
pd.options.display.float_format = '{:,.4f}'.format

dfODSummary_pivotMode_OPct = dfODSummary.pivot_table(index=('group','OID','DID','date_range','day_type','day_part'),values=('OPercentTotal'),columns='mode')
dfODSummary_pivotMode_OPct = dfODSummary_pivotMode_OPct.reset_index()
dfODSummary_pivotMode_OPct = dfODSummary_pivotMode_OPct.fillna(0)
dfODSummary_pivotMode_OPct = dfODSummary_pivotMode_OPct.rename(columns={'OID':'O','DID':'D','ALL':"AO",'BIC':'KO','BUS':'SO','PED':'PO','RAI':'RO','TRU':'TO'})
display(dfODSummary_pivotMode_OPct)

dfODSummary_pivotMode_DPct = dfODSummary.pivot_table(index=('group','OID','DID','date_range','day_type','day_part'),values=('DPercentTotal'),columns='mode')
dfODSummary_pivotMode_DPct = dfODSummary_pivotMode_DPct.reset_index()
dfODSummary_pivotMode_DPct = dfODSummary_pivotMode_DPct.fillna(0)
dfODSummary_pivotMode_DPct = dfODSummary_pivotMode_DPct.rename(columns={'OID':'O','DID':'D','ALL':"AD",'BIC':'KD','BUS':'SD','PED':'PD','RAI':'RD','TRU':'TD'})
display(dfODSummary_pivotMode_DPct)

dfODSummary_pivotMode_BPct = dfODSummary.pivot_table(index=('group','OID','DID','date_range','day_type','day_part'),values=('BPercentTotal'),columns='mode')
dfODSummary_pivotMode_BPct = dfODSummary_pivotMode_BPct.reset_index()
dfODSummary_pivotMode_BPct = dfODSummary_pivotMode_BPct.fillna(0)
dfODSummary_pivotMode_BPct = dfODSummary_pivotMode_BPct.rename(columns={'OID':'O','DID':'D','ALL':"AB",'BIC':'KB','BUS':'SB','PED':'PB','RAI':'RB','TRU':'TB'})
display(dfODSummary_pivotMode_BPct)

mode,group,O,D,date_range,day_type,day_part,AV,KV,SV,PV,RV,TV
0,centertosmalldist,11,2000,2019_03,0,0,1,0,0,0,0,4
1,centertosmalldist,11,2000,2019_03,0,2,1,0,0,0,0,0
2,centertosmalldist,11,2000,2019_03,0,4,0,0,0,0,0,4
3,centertosmalldist,11,2000,2019_03,1,0,0,0,0,0,0,8
4,centertosmalldist,11,2000,2019_03,1,4,0,0,0,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...
7917518,smalldist,721,701,2021_BM,2,4,1,0,0,0,0,0
7917519,smalldist,721,701,2021_YR,0,0,0,0,0,0,0,0
7917520,smalldist,721,701,2021_YR,0,4,0,0,0,0,0,0
7917521,smalldist,721,701,2021_YR,2,0,1,0,0,0,0,0


mode,group,O,D,date_range,day_type,day_part,AO,KO,SO,PO,RO,TO
0,centertosmalldist,11,2000,2019_03,0,0,0.0167,0.0000,0.0000,0.0000,0.0000,0.0367
1,centertosmalldist,11,2000,2019_03,0,2,0.0833,0.0000,0.0000,0.0000,0.0000,0.0000
2,centertosmalldist,11,2000,2019_03,0,4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0625
3,centertosmalldist,11,2000,2019_03,1,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0541
4,centertosmalldist,11,2000,2019_03,1,4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0976
...,...,...,...,...,...,...,...,...,...,...,...,...
7917518,smalldist,721,701,2021_BM,2,4,0.0022,0.0000,0.0000,0.0000,0.0000,0.0000
7917519,smalldist,721,701,2021_YR,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7917520,smalldist,721,701,2021_YR,0,4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7917521,smalldist,721,701,2021_YR,2,0,0.0010,0.0000,0.0000,0.0000,0.0000,0.0000


mode,group,O,D,date_range,day_type,day_part,AD,KD,SD,PD,RD,TD
0,centertosmalldist,11,2000,2019_03,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0002
1,centertosmalldist,11,2000,2019_03,0,2,0.0001,0.0000,0.0000,0.0000,0.0000,0.0000
2,centertosmalldist,11,2000,2019_03,0,4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0011
3,centertosmalldist,11,2000,2019_03,1,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0003
4,centertosmalldist,11,2000,2019_03,1,4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0015
...,...,...,...,...,...,...,...,...,...,...,...,...
7917518,smalldist,721,701,2021_BM,2,4,0.0204,0.0000,0.0000,0.0000,0.0000,0.0000
7917519,smalldist,721,701,2021_YR,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7917520,smalldist,721,701,2021_YR,0,4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7917521,smalldist,721,701,2021_YR,2,0,0.0078,0.0000,0.0000,0.0000,0.0000,0.0000


mode,group,O,D,date_range,day_type,day_part,AB,KB,SB,PB,RB,TB
0,centertosmalldist,11,2000,2019_03,0,0,0.0083,0.0000,0.0000,0.0000,0.0000,0.0185
1,centertosmalldist,11,2000,2019_03,0,2,0.0417,0.0000,0.0000,0.0000,0.0000,0.0000
2,centertosmalldist,11,2000,2019_03,0,4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0318
3,centertosmalldist,11,2000,2019_03,1,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0272
4,centertosmalldist,11,2000,2019_03,1,4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0496
...,...,...,...,...,...,...,...,...,...,...,...,...
7917518,smalldist,721,701,2021_BM,2,4,0.0113,0.0000,0.0000,0.0000,0.0000,0.0000
7917519,smalldist,721,701,2021_YR,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7917520,smalldist,721,701,2021_YR,0,4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7917521,smalldist,721,701,2021_YR,2,0,0.0044,0.0000,0.0000,0.0000,0.0000,0.0000


In [61]:
dfODSummary_pivotMode_Cmb = pd.DataFrame.merge(dfODSummary_pivotMode_Vol, dfODSummary_pivotMode_OPct, on=('group','O','D','date_range','day_type','day_part'), how='left')
dfODSummary_pivotMode_Cmb = pd.DataFrame.merge(dfODSummary_pivotMode_Cmb, dfODSummary_pivotMode_DPct, on=('group','O','D','date_range','day_type','day_part'), how='left')
dfODSummary_pivotMode_Cmb = pd.DataFrame.merge(dfODSummary_pivotMode_Cmb, dfODSummary_pivotMode_BPct, on=('group','O','D','date_range','day_type','day_part'), how='left')
dfODSummary_pivotMode_Cmb['O'] = dfODSummary_pivotMode_Cmb['O'].apply(lambda x: '{0:0>4}'.format(x))
dfODSummary_pivotMode_Cmb['D'] = dfODSummary_pivotMode_Cmb['D'].apply(lambda x: '{0:0>4}'.format(x))
dfODSummary_pivotMode_Cmb['I'] = dfODSummary_pivotMode_Cmb['O'].astype(str) + '_' + dfODSummary_pivotMode_Cmb['D'].astype(str)
dfODSummary_pivotMode_Cmb = dfODSummary_pivotMode_Cmb.drop(columns=['O','D'])
#dfODSummary_pivotMode_Cmb = dfODSummary_pivotMode_Cmb.set_index('I')
dfODSummary_pivotMode_Cmb


mode,group,date_range,day_type,day_part,AV,KV,SV,PV,RV,TV,...,PD,RD,TD,AB,KB,SB,PB,RB,TB,I
0,centertosmalldist,2019_03,0,0,1,0,0,0,0,4,...,0.0000,0.0000,0.0002,0.0083,0.0000,0.0000,0.0000,0.0000,0.0185,0011_2000
1,centertosmalldist,2019_03,0,2,1,0,0,0,0,0,...,0.0000,0.0000,0.0000,0.0417,0.0000,0.0000,0.0000,0.0000,0.0000,0011_2000
2,centertosmalldist,2019_03,0,4,0,0,0,0,0,4,...,0.0000,0.0000,0.0011,0.0000,0.0000,0.0000,0.0000,0.0000,0.0318,0011_2000
3,centertosmalldist,2019_03,1,0,0,0,0,0,0,8,...,0.0000,0.0000,0.0003,0.0000,0.0000,0.0000,0.0000,0.0000,0.0272,0011_2000
4,centertosmalldist,2019_03,1,4,0,0,0,0,0,8,...,0.0000,0.0000,0.0015,0.0000,0.0000,0.0000,0.0000,0.0000,0.0496,0011_2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7917518,smalldist,2021_BM,2,4,1,0,0,0,0,0,...,0.0000,0.0000,0.0000,0.0113,0.0000,0.0000,0.0000,0.0000,0.0000,0721_0701
7917519,smalldist,2021_YR,0,0,0,0,0,0,0,0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0721_0701
7917520,smalldist,2021_YR,0,4,0,0,0,0,0,0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0721_0701
7917521,smalldist,2021_YR,2,0,1,0,0,0,0,0,...,0.0000,0.0000,0.0000,0.0044,0.0000,0.0000,0.0000,0.0000,0.0000,0721_0701


In [62]:
dfFiles = dfODSummary_pivotMode_Cmb.groupby(['group','date_range','day_type','day_part'],as_index=False).agg(NUMREC=('I','count'))

for index, row in dfFiles.iterrows():
    #print(row['date_range'])
    dfExport = dfODSummary_pivotMode_Cmb[(dfODSummary_pivotMode_Cmb['date_range']==row['date_range']) & (dfODSummary_pivotMode_Cmb['day_type']==row['day_type']) & (dfODSummary_pivotMode_Cmb['day_part']==row['day_part'])]
    dfExport = dfExport.drop(columns=['group','date_range','day_part','day_type'])
    dfExport = dfExport.set_index('I')
    dfExport.to_json(os.path.join('results', row['group'] + '_' + str(row['date_range']) + '_' + str(row['day_type']) + '_' + str(row['day_part']) + '.json'),orient='index')

In [63]:
dfExport

mode,AV,KV,SV,PV,RV,TV,AO,KO,SO,PO,...,SD,PD,RD,TD,AB,KB,SB,PB,RB,TB
I,,,,,,,,,,,,,,,,,,,,,
0011_2001,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
0011_2002,1,0,0,0,0,0,0.0714,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0358,0.0000,0.0000,0.0000,0.0000,0.0000
0011_2003,1,0,0,0,0,0,0.0714,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0358,0.0000,0.0000,0.0000,0.0000,0.0000
0011_2005,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
0011_2008,0,0,0,0,0,5,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.5357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0721_0611,6,0,0,0,0,0,0.0300,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0171,0.0000,0.0000,0.0000,0.0000,0.0000
0721_0612,42,0,0,22,0,0,0.2100,0.0000,0.0000,0.5500,...,0.0000,0.0553,0.0000,0.0000,0.1079,0.0000,0.0000,0.3027,0.0000,0.0000
0721_0621,93,3,0,18,0,0,0.4650,1.0000,0.0000,0.4500,...,0.0000,0.0786,0.0000,0.0000,0.2394,0.5652,0.0000,0.2643,0.0000,0.0000
